In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pyspark.sql import SparkSession
from pyspark.sql.functions import stddev, percentile_approx, min, max, mean

In [2]:
spark = SparkSession.builder\
    .master("local[*]")\
    .config("spark.executor.memory", "8g")\
    .config("spark.driver.memory", "64g")\
    .getOrCreate()

22/08/24 18:40:40 WARN Utils: Your hostname, intellerts-HP-Z8-G4-Workstation resolves to a loopback address: 127.0.1.1; using 192.168.1.101 instead (on interface enp9s0f2)
22/08/24 18:40:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/08/24 18:40:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df = spark.read.options(header=True).csv('data/train_data.csv')
df = df.repartition(256, 'customer_ID')
labels = spark.read.options(header=True).csv('data/train_labels.csv')
value_cols = list(set(df.columns) - {'customer_ID', 'S_2', 'D_63', 'D_64'})

In [4]:
na_stats = {col:df.filter(df[col].isNull()).count() for col in df.columns}
na_stats = pd.DataFrame.from_records(list(na_stats.items()), columns=["Column", "NA count"])

In [5]:
aggs = [min(col).alias(f"min_{col}") for col in value_cols] + \
    [max(col).alias(f"max_{col}") for col in value_cols] + \
    [stddev(col).alias(f"std_{col}") for col in value_cols] + \
    [mean(col).alias(f"mean_{col}") for col in value_cols] + \
    [percentile_approx(col, 0.25).alias(f"perc25_{col}") for col in value_cols] + \
    [percentile_approx(col, 0.5).alias(f"median_{col}") for col in value_cols] + \
    [percentile_approx(col, 0.75).alias(f"perc75_{col}") for col in value_cols]

In [6]:
for col in tqdm(value_cols):
    df = df.withColumn(col, df[col].cast('double'))
grouped_df = df.groupBy('customer_ID')
aggs = grouped_df.agg(*aggs)
feats = aggs.join(labels, ['customer_ID'])
feats.write.parquet('features-spark.parquet')

22/08/23 15:45:28 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
22/08/23 15:45:41 WARN DAGScheduler: Broadcasting large task binary with size 1220.7 KiB


In [7]:
# Process test data
df = spark.read.options(header=True).csv('data/test_data.csv')
df = df.repartition(512, 'customer_ID')
for col in tqdm(value_cols):
    df = df.withColumn(col, df[col].cast('double'))
grouped_df = df.groupBy('customer_ID')
feats = grouped_df.agg(*aggs)
feats.write.parquet('features-spark-test.parquet')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 186/186 [00:04<00:00, 44.10it/s]
22/08/24 18:51:56 WARN DAGScheduler: Broadcasting large task binary with size 1206.1 KiB
